## Real Estate Data Transformation

In [1]:
import json
import pandas as pd
import numpy as np

### Get ids properties

In [2]:
import aiohttp
import asyncio

import nest_asyncio
nest_asyncio.apply()

async def get(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            data = json.loads(await response.text())
            return data['data']

loop = asyncio.get_event_loop()

result = loop.run_until_complete(get('https://api.arrendamientosnutibara.com/promotion/all-promotions'))

In [3]:
properties = [x for x in result if x['status'] == 'PROMOCION' and len(x['property']['images']) >= 3]

In [4]:
df = pd.DataFrame(properties)
df.drop(columns = ['sellValue', 'rentValue', 'status', 'situation', 'keys', 'property'], inplace = True)
ids = df.to_dict('records')
ids_list = []
for data in ids:
    ids_list.append(data['id'])

In [5]:
len(ids_list)

588

### Get data per property

In [6]:
import asyncio
from aiohttp import ClientSession

In [7]:
async def fetch(url, session):
    async with session.get(url) as response:
        response = json.loads(await response.text())
        return response['data']

In [8]:
properties_list = []

In [10]:
async def run(ids_list):
    url = "https://api.arrendamientosnutibara.com/promotion/{}"
    properties = []
    async with ClientSession() as session:
        for id in ids_list:
            property = asyncio.ensure_future(fetch(url.format(id), session))
            properties.append(property)

        responses = await asyncio.gather(*properties)
        for property in responses:
            properties_list.append(property)

In [11]:
loop = asyncio.get_event_loop()
future = asyncio.ensure_future(run(ids_list))
loop.run_until_complete(future)

In [12]:
len(properties_list)

588

### Data Cleaning

In [13]:
df_promotion = pd.DataFrame(properties_list)

In [14]:
df_promotion.drop(columns=['sellValue', 'status', 'keys'], inplace = True)

In [15]:
df_promotion.columns = ['promotion_id', 'promotion_rent', 'property']

In [16]:
promotion_detail = df_promotion['property']

In [17]:
df_promotion_detail = pd.DataFrame(list(promotion_detail))

In [18]:
df_promotion_detail = df_promotion_detail.rename(columns={
    "id": "property_id",
})

In [19]:
df_promotion.drop(columns = ['property'], inplace = True)

### Join DataFrames

In [20]:
df = df_promotion.join(df_promotion_detail)

### Replace words in property type

In [21]:
df['type'] = df['type'].str.replace(' SIMP', '')

### Capitalize

In [22]:
df['type'] = df['type'].str.capitalize()
df['neighborhood'] = df['neighborhood'].str.capitalize()
df['city'] = df['city'].str.capitalize()

### Delete columns Appropiate API doesn't need

In [23]:
df = df.drop(columns = ['sector'])

### Strip

In [24]:
df['type'] = df['type'].str.strip()

In [25]:
def lower_columns(df, cols):
    df[cols] = df[cols].apply(lambda x: x.str.lower())
    return df

In [26]:
lower_columns(df, ['type', 'neighborhood', 'city'])

,promotion_id,promotion_rent,property_id,type,neighborhood,city,latitude,longitude,stratum,images,facilities
0,45975,500000,17957,apartamento,caribe,medellin,6.273039,-75.572224,3,"[{'id': 5, 'resource': 'a861db299cd5979816d1ca...","[{'id': 525808, 'value': '2', 'facility': {'id..."
1,47237,680000,28978,local,centro,medellin,6.256035,-75.571223,3,"[{'id': 14, 'resource': 'a861db299cd5979816d1c...","[{'id': 558795, 'value': '2', 'facility': {'id..."
2,50320,850000,21007,casa,francisco antonio zea,medellin,6.286610,-75.573299,3,"[{'id': 40266, 'resource': '79de9c8c1e9dd76919...","[{'id': 608957, 'value': '3', 'facility': {'id..."
3,50833,1000000,8446,local,itagui,itagui,6.190032,-75.597829,3,"[{'id': 139, 'resource': '3f1fbec263afbcb6873b...","[{'id': 506111, 'value': '0', 'facility': {'id..."
4,51352,8500000,28820,local,centro,medellin,6.252109,-75.567265,4,"[{'id': 195, 'resource': 'a861db299cd5979816d1...","[{'id': 557546, 'value': '0', 'facility': {'id..."
...,...,...,...,...,...,...,...,...,...,...,...
583,62877,800000,13613,apartamento,tricentenario,medellin,6.291108,-75.565718,3,"[{'id': 38315, 'resource': 'dbd6737a156da45c18...","[{'id': 513777, 'value': '3', 'facility': {'id..."
584,62879,1900000,34419,casa,laureles,medellin,6.238079,-75.594181,5,"[{'id': 123487, 'resource': 'a1f28476c61a12a50...","[{'id': 625083, 'value': '3', 'facility': {'id..."
585,62884,2200000,30704,local,centro,medellin,6.253478,-75.562653,4,"[{'id': 4134, 'resource': 'b0826b182a028380f33...","[{'id': 569971, 'value': '2', 'facility': {'id..."
586,62886,2500000,34422,casa,estadio,medellin,6.253917,-75.582674,5,"[{'id': 123390, 'resource': 'ba06c381ed267fb7d...","[{'id': 625183, 'value': '0', 'facility': {'id..."


In [27]:
def remove_accents(df, cols):
    df[cols] = df[cols].apply(lambda x: x.str.replace("Ñ", "%"))
    df[cols] = df[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('ascii'))
    df[cols] = df[cols].apply(lambda x: x.str.replace("%", "Ñ"))
    return df

In [28]:
remove_accents(df, ['city', 'type', 'neighborhood'])

,promotion_id,promotion_rent,property_id,type,neighborhood,city,latitude,longitude,stratum,images,facilities
0,45975,500000,17957,apartamento,caribe,medellin,6.273039,-75.572224,3,"[{'id': 5, 'resource': 'a861db299cd5979816d1ca...","[{'id': 525808, 'value': '2', 'facility': {'id..."
1,47237,680000,28978,local,centro,medellin,6.256035,-75.571223,3,"[{'id': 14, 'resource': 'a861db299cd5979816d1c...","[{'id': 558795, 'value': '2', 'facility': {'id..."
2,50320,850000,21007,casa,francisco antonio zea,medellin,6.286610,-75.573299,3,"[{'id': 40266, 'resource': '79de9c8c1e9dd76919...","[{'id': 608957, 'value': '3', 'facility': {'id..."
3,50833,1000000,8446,local,itagui,itagui,6.190032,-75.597829,3,"[{'id': 139, 'resource': '3f1fbec263afbcb6873b...","[{'id': 506111, 'value': '0', 'facility': {'id..."
4,51352,8500000,28820,local,centro,medellin,6.252109,-75.567265,4,"[{'id': 195, 'resource': 'a861db299cd5979816d1...","[{'id': 557546, 'value': '0', 'facility': {'id..."
...,...,...,...,...,...,...,...,...,...,...,...
583,62877,800000,13613,apartamento,tricentenario,medellin,6.291108,-75.565718,3,"[{'id': 38315, 'resource': 'dbd6737a156da45c18...","[{'id': 513777, 'value': '3', 'facility': {'id..."
584,62879,1900000,34419,casa,laureles,medellin,6.238079,-75.594181,5,"[{'id': 123487, 'resource': 'a1f28476c61a12a50...","[{'id': 625083, 'value': '3', 'facility': {'id..."
585,62884,2200000,30704,local,centro,medellin,6.253478,-75.562653,4,"[{'id': 4134, 'resource': 'b0826b182a028380f33...","[{'id': 569971, 'value': '2', 'facility': {'id..."
586,62886,2500000,34422,casa,estadio,medellin,6.253917,-75.582674,5,"[{'id': 123390, 'resource': 'ba06c381ed267fb7d...","[{'id': 625183, 'value': '0', 'facility': {'id..."


In [29]:
def convert_to_int_columns(df, cols):
    df[cols] = df[cols].apply(lambda x: x.astype(int))
    return df

In [30]:
convert_to_int_columns(df, ['stratum'])

,promotion_id,promotion_rent,property_id,type,neighborhood,city,latitude,longitude,stratum,images,facilities
0,45975,500000,17957,apartamento,caribe,medellin,6.273039,-75.572224,3,"[{'id': 5, 'resource': 'a861db299cd5979816d1ca...","[{'id': 525808, 'value': '2', 'facility': {'id..."
1,47237,680000,28978,local,centro,medellin,6.256035,-75.571223,3,"[{'id': 14, 'resource': 'a861db299cd5979816d1c...","[{'id': 558795, 'value': '2', 'facility': {'id..."
2,50320,850000,21007,casa,francisco antonio zea,medellin,6.286610,-75.573299,3,"[{'id': 40266, 'resource': '79de9c8c1e9dd76919...","[{'id': 608957, 'value': '3', 'facility': {'id..."
3,50833,1000000,8446,local,itagui,itagui,6.190032,-75.597829,3,"[{'id': 139, 'resource': '3f1fbec263afbcb6873b...","[{'id': 506111, 'value': '0', 'facility': {'id..."
4,51352,8500000,28820,local,centro,medellin,6.252109,-75.567265,4,"[{'id': 195, 'resource': 'a861db299cd5979816d1...","[{'id': 557546, 'value': '0', 'facility': {'id..."
...,...,...,...,...,...,...,...,...,...,...,...
583,62877,800000,13613,apartamento,tricentenario,medellin,6.291108,-75.565718,3,"[{'id': 38315, 'resource': 'dbd6737a156da45c18...","[{'id': 513777, 'value': '3', 'facility': {'id..."
584,62879,1900000,34419,casa,laureles,medellin,6.238079,-75.594181,5,"[{'id': 123487, 'resource': 'a1f28476c61a12a50...","[{'id': 625083, 'value': '3', 'facility': {'id..."
585,62884,2200000,30704,local,centro,medellin,6.253478,-75.562653,4,"[{'id': 4134, 'resource': 'b0826b182a028380f33...","[{'id': 569971, 'value': '2', 'facility': {'id..."
586,62886,2500000,34422,casa,estadio,medellin,6.253917,-75.582674,5,"[{'id': 123390, 'resource': 'ba06c381ed267fb7d...","[{'id': 625183, 'value': '0', 'facility': {'id..."


In [31]:
def convert_to_float_columns(df, cols):
    df[cols] = df[cols].apply(lambda x: x.astype(float))
    return df

In [32]:
convert_to_float_columns(df, ['stratum'])

,promotion_id,promotion_rent,property_id,type,neighborhood,city,latitude,longitude,stratum,images,facilities
0,45975,500000,17957,apartamento,caribe,medellin,6.273039,-75.572224,3.0,"[{'id': 5, 'resource': 'a861db299cd5979816d1ca...","[{'id': 525808, 'value': '2', 'facility': {'id..."
1,47237,680000,28978,local,centro,medellin,6.256035,-75.571223,3.0,"[{'id': 14, 'resource': 'a861db299cd5979816d1c...","[{'id': 558795, 'value': '2', 'facility': {'id..."
2,50320,850000,21007,casa,francisco antonio zea,medellin,6.286610,-75.573299,3.0,"[{'id': 40266, 'resource': '79de9c8c1e9dd76919...","[{'id': 608957, 'value': '3', 'facility': {'id..."
3,50833,1000000,8446,local,itagui,itagui,6.190032,-75.597829,3.0,"[{'id': 139, 'resource': '3f1fbec263afbcb6873b...","[{'id': 506111, 'value': '0', 'facility': {'id..."
4,51352,8500000,28820,local,centro,medellin,6.252109,-75.567265,4.0,"[{'id': 195, 'resource': 'a861db299cd5979816d1...","[{'id': 557546, 'value': '0', 'facility': {'id..."
...,...,...,...,...,...,...,...,...,...,...,...
583,62877,800000,13613,apartamento,tricentenario,medellin,6.291108,-75.565718,3.0,"[{'id': 38315, 'resource': 'dbd6737a156da45c18...","[{'id': 513777, 'value': '3', 'facility': {'id..."
584,62879,1900000,34419,casa,laureles,medellin,6.238079,-75.594181,5.0,"[{'id': 123487, 'resource': 'a1f28476c61a12a50...","[{'id': 625083, 'value': '3', 'facility': {'id..."
585,62884,2200000,30704,local,centro,medellin,6.253478,-75.562653,4.0,"[{'id': 4134, 'resource': 'b0826b182a028380f33...","[{'id': 569971, 'value': '2', 'facility': {'id..."
586,62886,2500000,34422,casa,estadio,medellin,6.253917,-75.582674,5.0,"[{'id': 123390, 'resource': 'ba06c381ed267fb7d...","[{'id': 625183, 'value': '0', 'facility': {'id..."


In [33]:
def fillna_columns(df, cols, value):
    df[cols] = df[cols].apply(lambda x: x.fillna(value))
    return df

In [34]:
fillna_columns(df, ['latitude', 'longitude'], 0.0)

,promotion_id,promotion_rent,property_id,type,neighborhood,city,latitude,longitude,stratum,images,facilities
0,45975,500000,17957,apartamento,caribe,medellin,6.273039,-75.572224,3.0,"[{'id': 5, 'resource': 'a861db299cd5979816d1ca...","[{'id': 525808, 'value': '2', 'facility': {'id..."
1,47237,680000,28978,local,centro,medellin,6.256035,-75.571223,3.0,"[{'id': 14, 'resource': 'a861db299cd5979816d1c...","[{'id': 558795, 'value': '2', 'facility': {'id..."
2,50320,850000,21007,casa,francisco antonio zea,medellin,6.286610,-75.573299,3.0,"[{'id': 40266, 'resource': '79de9c8c1e9dd76919...","[{'id': 608957, 'value': '3', 'facility': {'id..."
3,50833,1000000,8446,local,itagui,itagui,6.190032,-75.597829,3.0,"[{'id': 139, 'resource': '3f1fbec263afbcb6873b...","[{'id': 506111, 'value': '0', 'facility': {'id..."
4,51352,8500000,28820,local,centro,medellin,6.252109,-75.567265,4.0,"[{'id': 195, 'resource': 'a861db299cd5979816d1...","[{'id': 557546, 'value': '0', 'facility': {'id..."
...,...,...,...,...,...,...,...,...,...,...,...
583,62877,800000,13613,apartamento,tricentenario,medellin,6.291108,-75.565718,3.0,"[{'id': 38315, 'resource': 'dbd6737a156da45c18...","[{'id': 513777, 'value': '3', 'facility': {'id..."
584,62879,1900000,34419,casa,laureles,medellin,6.238079,-75.594181,5.0,"[{'id': 123487, 'resource': 'a1f28476c61a12a50...","[{'id': 625083, 'value': '3', 'facility': {'id..."
585,62884,2200000,30704,local,centro,medellin,6.253478,-75.562653,4.0,"[{'id': 4134, 'resource': 'b0826b182a028380f33...","[{'id': 569971, 'value': '2', 'facility': {'id..."
586,62886,2500000,34422,casa,estadio,medellin,6.253917,-75.582674,5.0,"[{'id': 123390, 'resource': 'ba06c381ed267fb7d...","[{'id': 625183, 'value': '0', 'facility': {'id..."


### Extract images

In [35]:
data = df.to_dict('records')

In [36]:
from pandas.io.json import json_normalize

In [37]:
df_images = json_normalize(data, record_path='images', meta=['promotion_id', 'property_id'])

<ipython-input-37-a0b3269d8d9b>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df_images = json_normalize(data, record_path='images', meta=['promotion_id', 'property_id'])


In [38]:
df_images.drop(columns = ['id'], inplace = True)

In [39]:
df_images.head()

,resource,main,promotion_id,property_id
0,a861db299cd5979816d1cad6f6baaeb4.jpg,False,45975,17957
1,dd64b37838f389e5f61ed46194aec023.jpg,False,45975,17957
2,98d38ff7ac3cfba8c853cb4be8dbf2e0.jpg,False,45975,17957
3,0752117deeffb228abc311287ca3a764.jpg,False,45975,17957
4,09c1b3b1527f9164683f2ab55d850add.jpg,False,45975,17957


In [40]:
df_images['url'] = 'https://assets.arrendamientosnutibara.com/spaces/images/' + df_images['property_id'].astype(str) + '/720' + df_images['resource']

In [41]:
df_images = df_images.drop(columns = ['resource'])

In [42]:
df_images

,main,promotion_id,property_id,url
0,False,45975,17957,https://assets.arrendamientosnutibara.com/spac...
1,False,45975,17957,https://assets.arrendamientosnutibara.com/spac...
2,False,45975,17957,https://assets.arrendamientosnutibara.com/spac...
3,False,45975,17957,https://assets.arrendamientosnutibara.com/spac...
4,False,45975,17957,https://assets.arrendamientosnutibara.com/spac...
...,...,...,...,...
5224,False,62895,33373,https://assets.arrendamientosnutibara.com/spac...
5225,False,62895,33373,https://assets.arrendamientosnutibara.com/spac...
5226,False,62895,33373,https://assets.arrendamientosnutibara.com/spac...
5227,False,62895,33373,https://assets.arrendamientosnutibara.com/spac...


In [43]:
images = df_images.to_dict('records')

In [44]:
sorted_list = sorted(images, key=lambda x: x['main'] == True, reverse=True)

In [45]:
df_images = pd.DataFrame(sorted_list)

In [46]:
df_images

,main,promotion_id,property_id,url
0,True,45975,17957,https://assets.arrendamientosnutibara.com/spac...
1,True,47237,28978,https://assets.arrendamientosnutibara.com/spac...
2,True,50320,21007,https://assets.arrendamientosnutibara.com/spac...
3,True,50833,8446,https://assets.arrendamientosnutibara.com/spac...
4,True,51352,28820,https://assets.arrendamientosnutibara.com/spac...
...,...,...,...,...
5224,False,62895,33373,https://assets.arrendamientosnutibara.com/spac...
5225,False,62895,33373,https://assets.arrendamientosnutibara.com/spac...
5226,False,62895,33373,https://assets.arrendamientosnutibara.com/spac...
5227,False,62895,33373,https://assets.arrendamientosnutibara.com/spac...


In [47]:
df_images = df_images.groupby('promotion_id')['url'].apply(list).reset_index(name='images')

In [48]:
df = df.drop(columns = ['images'])

In [49]:
df = pd.merge(df, df_images)

In [50]:
df

,promotion_id,promotion_rent,property_id,type,neighborhood,city,latitude,longitude,stratum,facilities,images
0,45975,500000,17957,apartamento,caribe,medellin,6.273039,-75.572224,3.0,"[{'id': 525808, 'value': '2', 'facility': {'id...",[https://assets.arrendamientosnutibara.com/spa...
1,47237,680000,28978,local,centro,medellin,6.256035,-75.571223,3.0,"[{'id': 558795, 'value': '2', 'facility': {'id...",[https://assets.arrendamientosnutibara.com/spa...
2,50320,850000,21007,casa,francisco antonio zea,medellin,6.286610,-75.573299,3.0,"[{'id': 608957, 'value': '3', 'facility': {'id...",[https://assets.arrendamientosnutibara.com/spa...
3,50833,1000000,8446,local,itagui,itagui,6.190032,-75.597829,3.0,"[{'id': 506111, 'value': '0', 'facility': {'id...",[https://assets.arrendamientosnutibara.com/spa...
4,51352,8500000,28820,local,centro,medellin,6.252109,-75.567265,4.0,"[{'id': 557546, 'value': '0', 'facility': {'id...",[https://assets.arrendamientosnutibara.com/spa...
...,...,...,...,...,...,...,...,...,...,...,...
583,62877,800000,13613,apartamento,tricentenario,medellin,6.291108,-75.565718,3.0,"[{'id': 513777, 'value': '3', 'facility': {'id...",[https://assets.arrendamientosnutibara.com/spa...
584,62879,1900000,34419,casa,laureles,medellin,6.238079,-75.594181,5.0,"[{'id': 625083, 'value': '3', 'facility': {'id...",[https://assets.arrendamientosnutibara.com/spa...
585,62884,2200000,30704,local,centro,medellin,6.253478,-75.562653,4.0,"[{'id': 569971, 'value': '2', 'facility': {'id...",[https://assets.arrendamientosnutibara.com/spa...
586,62886,2500000,34422,casa,estadio,medellin,6.253917,-75.582674,5.0,"[{'id': 625183, 'value': '0', 'facility': {'id...",[https://assets.arrendamientosnutibara.com/spa...


### Extract facilities

In [51]:
df_facilities = json_normalize(data, record_path='facilities', meta=['promotion_id', 'property_id'])

<ipython-input-51-409763e04d99>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df_facilities = json_normalize(data, record_path='facilities', meta=['promotion_id', 'property_id'])


In [52]:
df_facilities = df_facilities.drop(columns = ['id'])

In [53]:
df_facilities.columns

Index(['value', 'facility.id', 'facility.name', 'promotion_id', 'property_id'], dtype='object')

In [54]:
df_facilities = df_facilities.rename(columns = {
    'facility.id': 'facility_id',
    'value': 'facility_value',
    'facility.name': 'facility_name'
})

In [55]:
df_facilities

,facility_value,facility_id,facility_name,promotion_id,property_id
0,2,82,Alcobas,45975,17957
1,0,93,Salon Comedor,45975,17957
2,0,96,Trifilar,45975,17957
3,1,99,Balcon,45975,17957
4,0,101,Patios,45975,17957
...,...,...,...,...,...
7389,0,119,Piso Baldosa,62895,33373
7390,0,129,Rejas,62895,33373
7391,0,130,Red Gas,62895,33373
7392,0,131,Cocina Sencilla,62895,33373


### Facilities

##### 112 - Área
##### 105 - Baños
##### 82 - Habitaciones

### Extract rooms

In [56]:
df_rooms = df_facilities.loc[df_facilities['facility_id'] == 82]

In [57]:
df_rooms

,facility_value,facility_id,facility_name,promotion_id,property_id
0,2,82,Alcobas,45975,17957
12,2,82,Alcobas,47237,28978
23,3,82,Alcobas,50320,21007
60,3,82,Alcobas,52782,28721
71,3,82,Alcobas,52783,28720
...,...,...,...,...,...
7330,3,82,Alcobas,62877,13613
7344,3,82,Alcobas,62879,34419
7360,2,82,Alcobas,62884,30704
7370,3,82,Alcobas,62886,34422


In [58]:
df_rooms = df_rooms.drop(columns = ['property_id', 'facility_id', 'facility_name'])
df_rooms['facility_value'] = df_rooms['facility_value'].astype(int)

In [59]:
df_rooms.columns

Index(['facility_value', 'promotion_id'], dtype='object')

In [60]:
df_rooms = df_rooms.rename(columns={
    "facility_value": "rooms",
})

In [61]:
df_rooms

,rooms,promotion_id
0,2,45975
12,2,47237
23,3,50320
60,3,52782
71,3,52783
...,...,...
7330,3,62877
7344,3,62879
7360,2,62884
7370,3,62886


In [62]:
df['rooms'] = df.promotion_id.map(df_rooms.set_index('promotion_id')['rooms'])
df['rooms'] = df['rooms'].fillna(0)
df['rooms'] = df['rooms'].astype(int)

### Extract area

In [63]:
is_area = df_facilities['facility_id'] == 112
df_area = df_facilities[is_area]
df_area = df_area.drop(columns = ['property_id', 'facility_id', 'facility_name'])
df_area['facility_value'] = df_area['facility_value'].str.replace(',', '.')
df_area['facility_value'] = df_area['facility_value'].astype(float)
df_area = df_area.rename(columns={
    "facility_value": "area",
})

In [64]:
df_area

,area,promotion_id
10,45.0,45975
22,65.0,47237
35,108.0,50320
40,25.0,50833
45,700.0,51352
...,...,...
7328,62.0,62876
7339,68.0,62877
7365,130.0,62884
7378,180.0,62886


In [65]:
df['area'] = df.promotion_id.map(df_area.set_index('promotion_id')['area'])
df['area'] = df['area'].fillna('')

### Extract bathrooms

In [66]:
is_bath = df_facilities['facility_id'] == 105
df_bath = df_facilities[is_bath]
df_bath = df_bath.drop(columns = ['property_id', 'facility_id', 'facility_name'])
df_bath['facility_value'] = df_bath['facility_value'].astype(int)
df_bath = df_bath.rename(columns={
    "facility_value": "bathrooms",
})

In [67]:
df_bath

,bathrooms,promotion_id
8,1,45975
20,2,47237
33,1,50320
38,1,50833
43,5,51352
...,...,...
7336,1,62877
7352,3,62879
7364,2,62884
7377,2,62886


In [68]:
df['bathrooms'] = df.promotion_id.map(df_bath.set_index('promotion_id')['bathrooms'])
df['bathrooms'] = df['bathrooms'].fillna(0)
df['bathrooms'] = df['bathrooms'].astype(int)

In [69]:
df_facilities

,facility_value,facility_id,facility_name,promotion_id,property_id
0,2,82,Alcobas,45975,17957
1,0,93,Salon Comedor,45975,17957
2,0,96,Trifilar,45975,17957
3,1,99,Balcon,45975,17957
4,0,101,Patios,45975,17957
...,...,...,...,...,...
7389,0,119,Piso Baldosa,62895,33373
7390,0,129,Rejas,62895,33373
7391,0,130,Red Gas,62895,33373
7392,0,131,Cocina Sencilla,62895,33373


In [70]:
df_facilities = df_facilities[df_facilities.facility_id != 82]
df_facilities = df_facilities[df_facilities.facility_id != 105]
df_facilities = df_facilities[df_facilities.facility_id != 112]

### Extract other facilities

In [71]:
df_facilities = df_facilities.groupby('promotion_id')['facility_name'].apply(list).reset_index(name='facilities')

In [72]:
df = df.drop(columns = ['facilities'])

In [73]:
df = pd.merge(df, df_facilities)

### Convert to lower

In [74]:
df['city'] = df['city'].str.lower()
df['type'] = df['type'].str.lower()
df['neighborhood'] = df['neighborhood'].str.lower()

### Remove accents

In [75]:
cols = ['city', 'type', 'neighborhood']
df[cols] = df[cols].apply(lambda x: x.str.replace("Ñ", "%"))
df[cols] = df[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('ascii'))
df[cols] = df[cols].apply(lambda x: x.str.replace("%", "Ñ"))

### Get type properties

In [76]:
def get_type_property(case):
   sw = {
      "apartaestudio": 38,
      "apartamento": 2,
      "bodega": 27,
      "casa": 33,
      "casa local": 40,
      "consultorio": 29,
      "edificio": 30,
      "finca": 39, # Finca en Parcelacion
      "finca productiva": 31,
      "finca recreativa": 35,
      "hotel/apart hotel": 32,
      "local": 26,
      "lote comercial": 28,
      "lote": 37, # Lote en Parcelacion
      "lote independiente": 34,
      "lote industrial": 36,
      "oficina": 25
   }
   return sw.get(case, 'Invalid option')

In [77]:
df['type'] = df['type'].apply(lambda x: get_type_property(x))

In [78]:
df['type'] = df['type'].astype(int)

In [79]:
df

,promotion_id,promotion_rent,property_id,type,neighborhood,city,latitude,longitude,stratum,images,rooms,area,bathrooms,facilities
0,45975,500000,17957,2,caribe,medellin,6.273039,-75.572224,3.0,[https://assets.arrendamientosnutibara.com/spa...,2,45.0,1,"[Salon Comedor, Trifilar, Balcon, Patios, Zona..."
1,47237,680000,28978,26,centro,medellin,6.256035,-75.571223,3.0,[https://assets.arrendamientosnutibara.com/spa...,2,65.0,2,"[Closet, Salon Comedor, Balcon, Patios, Zona R..."
2,50320,850000,21007,33,francisco antonio zea,medellin,6.286610,-75.573299,3.0,[https://assets.arrendamientosnutibara.com/spa...,3,108.0,1,"[Closet, Salon Comedor, Cocina Sencilla, Balco..."
3,50833,1000000,8446,26,itagui,itagui,6.190032,-75.597829,3.0,[https://assets.arrendamientosnutibara.com/spa...,0,25.0,1,"[Barra Americana, Lavamanos, Red Gas]"
4,51352,8500000,28820,26,centro,medellin,6.252109,-75.567265,4.0,[https://assets.arrendamientosnutibara.com/spa...,0,700.0,5,"[Un Ambiente, Lavamanos, Ascensor, Porteria, P..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583,62877,800000,13613,2,tricentenario,medellin,6.291108,-75.565718,3.0,[https://assets.arrendamientosnutibara.com/spa...,3,68.0,1,"[Closet, Salon Comedor, Zona Ropa, Lavadero, I..."
584,62879,1900000,34419,33,laureles,medellin,6.238079,-75.594181,5.0,[https://assets.arrendamientosnutibara.com/spa...,3,,3,"[Closet, Sala, Comedor, Cuarto Util, Patios, L..."
585,62884,2200000,30704,26,centro,medellin,6.253478,-75.562653,4.0,[https://assets.arrendamientosnutibara.com/spa...,2,130.0,2,"[Terraza, Patios, Lavadero, Salon Social, Nota 5]"
586,62886,2500000,34422,33,estadio,medellin,6.253917,-75.582674,5.0,[https://assets.arrendamientosnutibara.com/spa...,3,180.0,2,"[Cocina Integral, Nota 5, Alcoba de servicio, ..."


### Get cities

In [81]:
async def get_cities(url):
    async with aiohttp.ClientSession(headers=headers) as session:
        async with session.get(url) as response:
            data = json.loads(await response.text())
            cities = data['response']
            return cities

loop = asyncio.get_event_loop()

cities = loop.run_until_complete(get_cities('http://appropiate.com/api/ciudad'))

In [82]:
df_cities = pd.DataFrame(list(cities))

In [83]:
cols = ['nombre']
df_cities[cols] = df_cities[cols].apply(lambda x: x.str.replace("Ñ", "%"))
df_cities[cols] = df_cities[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('ascii'))
df_cities[cols] = df_cities[cols].apply(lambda x: x.str.replace("%", "Ñ"))

In [84]:
df_cities['nombre'] = df_cities['nombre'].str.lower()

In [85]:
list_cities = df_cities.to_dict('records')

In [86]:
def get_city(case, cities):
    for city in cities:
        if(case in city['nombre']):
            return int(city['id'])

In [87]:
df['city'] = df['city'].apply(lambda x: get_city(x, list_cities))

### Get neighborhoods

In [89]:
async def get_neighborhoods(url):
    async with aiohttp.ClientSession(headers=headers) as session:
        async with session.get(url) as response:
            data = json.loads(await response.text())
            neighborhoods = data['response']
            return neighborhoods

loop = asyncio.get_event_loop()

neighborhoods = loop.run_until_complete(get_neighborhoods('http://appropiate.com/api/barrio'))

In [90]:
df_neighborhoods = pd.DataFrame(list(neighborhoods))

In [91]:
cols = ['nombre']
df_neighborhoods[cols] = df_neighborhoods[cols].apply(lambda x: x.str.replace("Ñ", "%"))
df_neighborhoods[cols] = df_neighborhoods[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('ascii'))
df_neighborhoods[cols] = df_neighborhoods[cols].apply(lambda x: x.str.replace("%", "Ñ"))

In [92]:
df_neighborhoods['nombre'] = df_neighborhoods['nombre'].str.lower()

In [93]:
list_neighborhoods = df_neighborhoods.to_dict('records')

In [94]:
def get_neighborhood(case, neighborhoods):
    for neighborhood in neighborhoods:
        if(case in neighborhood['nombre']):
            return int(neighborhood['id'])

In [95]:
df['neighborhood'] = df['neighborhood'].apply(lambda x: get_neighborhood(x, list_neighborhoods))
df['neighborhood'] = df['neighborhood'].fillna('')

### Get other facilities

In [97]:
async def get_facilities(url):
    async with aiohttp.ClientSession(headers=headers) as session:
        async with session.get(url) as response:
            data = json.loads(await response.text())
            facilities = data['response']
            return facilities

loop = asyncio.get_event_loop()

facilities = loop.run_until_complete(get_facilities('http://appropiate.com/api/caracteristica'))

In [98]:
df_facilities = pd.DataFrame(list(facilities))

In [99]:
cols = ['nombre']
df_facilities[cols] = df_facilities[cols].apply(lambda x: x.str.replace("Ñ", "%"))
df_facilities[cols] = df_facilities[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('ascii'))
df_facilities[cols] = df_facilities[cols].apply(lambda x: x.str.replace("%", "Ñ"))

In [100]:
df_facilities['nombre'] = df_facilities['nombre'].str.lower()

In [101]:
list_facilities = df_facilities.to_dict('records')

In [102]:
def map_other_facilities():
   other_facilities_dict = {
      "Alcoba Servicio": 83,
      "Biblioteca": 481,
      "Mezanine": 238,
      "Closet": 249,
      "Comedor": 443,
      "Salon Comedor": 477,
      "Barra Americana": 157,
      "CuartoUtil": 139,
      "Hall": 516,
      "Balcon": 133,
      "Terraza": 270,
      "Patios": 163,
      "Zona Ropa": 169,
      "Calentador": 200,
      "Unidad Cerrada": 319,
      "Juegos Infantiles": 390,
      "Ascensor": 355,
      "Zona Verde": 320,
      "Porteria": 327,
      "Piscina": 325,
      "Salon Social": 318,
      "Citofono": 201,
      "Aire Acondicionado": 180,
      "Shut Basura": 451,
      "Red Gas": 205,
      "Cocina Integral": 133,
      "Parqueadero Cubierto": 343,
      "Parqueadero Descubierto": 343,
      "Adm. 18. Libre ADM Of. Paga": 456,
      "Adm. 56. Libre ADM Pt. Paga": 456,
      "Adm. 38. Adm en Of. Of. Paga": 456,
      "Adm. 101 Adm en Of. Pt. Paga": 456,
      "Persiana": 243,
      "Cocineta": 258,
      "Gimnasio": 321,
      "Sauna": 323,
      "Turco": 324,
      "Cancha Sintetica": 273,
      "Cancha Polideportiva": 209,
      "Parqueadero Moto": 343,
      "Sotano": 248,
      "Circuito cerrado tv": 257,
      "Campestre": 227,
      "Con Local": 426,
      "jacuzzi": 300
   }
   return other_facilities_dict

### Map other facilities

In [103]:
data_dict = map_other_facilities()

In [104]:
df['facilities'] = df['facilities'].apply(lambda x: [data_dict.get(v) for v in x if v in data_dict])

In [105]:
df[df['promotion_rent'] == 0]

,promotion_id,promotion_rent,property_id,type,neighborhood,city,latitude,longitude,stratum,images,rooms,area,bathrooms,facilities


In [106]:
df[df['area'] == 0]

,promotion_id,promotion_rent,property_id,type,neighborhood,city,latitude,longitude,stratum,images,rooms,area,bathrooms,facilities


In [107]:
df[df['latitude'] == 0]

,promotion_id,promotion_rent,property_id,type,neighborhood,city,latitude,longitude,stratum,images,rooms,area,bathrooms,facilities
582,62876,700000,20644,2,1514,91,0.0,0.0,3.0,[https://assets.arrendamientosnutibara.com/spa...,2,62.0,1,"[249, 443, 157, 133, 169, 133]"


In [108]:
df[df['longitude'] == 0]

,promotion_id,promotion_rent,property_id,type,neighborhood,city,latitude,longitude,stratum,images,rooms,area,bathrooms,facilities
582,62876,700000,20644,2,1514,91,0.0,0.0,3.0,[https://assets.arrendamientosnutibara.com/spa...,2,62.0,1,"[249, 443, 157, 133, 169, 133]"


In [109]:
df[df['promotion_rent'] == 0]

,promotion_id,promotion_rent,property_id,type,neighborhood,city,latitude,longitude,stratum,images,rooms,area,bathrooms,facilities
